In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd

jar_files = [
    "/util/clickhouse-jdbc-0.3.2-all.jar"
]

# Initialize Spark session with JARs
spark = SparkSession.builder \
    .appName("combina_shapes") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", ",".join(jar_files)) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","admin") \
    .config("spark.hadoop.fs.s3a.secret.key","minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


url = "jdbc:ch://clickhouse:8123/cameraaberta"
user = "admin" 
password = "admin"  
driver = "com.clickhouse.jdbc.ClickHouseDriver"

display(spark)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: Can only call getServletHandlers on a running MetricsSystem
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.metrics.MetricsSystem.getServletHandlers(MetricsSystem.scala:89)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:650)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at py4j.re

In [ ]:
# df_previsao = pd.read_json(f'./previsao.json', ignore)
# print(df_previsao.shape)
# df_previsao.head()

In [ ]:
df_stops = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/stops.txt").toPandas()
# show data
df_stops.head(10)

In [ ]:
df_trips = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/trips.txt").toPandas()
# show data
df_trips.head(10)

In [ ]:
df_shapes = pd.read_csv('trips_stops.csv')
# show data
df_shapes.head(10)

In [ ]:
 list_trips

In [ ]:
import random


list_trips = df_trips['trip_id'].values

data = []

for s in df_shapes.values:
    data.append({"vs_ta" : 1, "trip_id":random.choice(list_trips),  'stop_id': s[6], 'vs_p': '1', 'shape_id': s[1] })
    
data[:10]
    

In [ ]:
df_chegadas = spark.createDataFrame(data)
df = spark.createDataFrame(df_chegadas)
df.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "chegadas") \
    .option("createTableOptions", "ENGINE=logs()") \
    .mode("overwrite") \
    .save()